In [10]:
import pandas as pd
import time
import datetime
import logging
import re
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

In [11]:
logging.basicConfig(filename='lol_data.log',  
                    level=logging.DEBUG)

In [12]:
#function that gets the html of all solo queue matches from league of Graphs of a given summoner
def lographs_htmlScrapper(summoner):
        button_css = 'button[class="see_more_ajax_button"]'
        driver = webdriver.Edge('C:\Git Projects\edgedriver_win64\msedgedriver.exe')
        driver.get("https://www.leagueofgraphs.com/summoner/lan/" + summoner)
        
        menu = driver.find_element_by_css_selector('a[data-dropdown="drop-queueTypes-recentgames"]')
        menu_item = driver.find_element_by_css_selector('a[data-additional-queue-id="440"]')
        ActionChains(driver).move_to_element(menu).click(menu).click(menu_item).perform()        #select option from menu
                    #while button exists get more matches 
    
        logging.debug("Getting data from the page..")
        while(True):         
            try:
                time.sleep(np.random.randint(2,3))
                button = driver.find_element_by_css_selector(button_css)              
                ActionChains(driver).move_to_element(button).pause(1.5).click(button).perform()
            except Exception as ex:
                
                print(f"{summoner}, Exception found {ex}")
                break
        page_source = driver.page_source
        driver.close()
        return page_source

In [13]:
#function that takes a list of summoner names and creates a list of dataframes for each summoner in the lis
def create_dataframe(summoners):
    match_history = []
    logging.debug("Creating a dataframe for each summoner...")
    for summoner in summoners:
            html = lographs_htmlScrapper(summoner) #getting the pagesource of the player
            matches = pd.read_html(html)[0] #there are many tables in the page, we only want the first
            matches = matches.iloc[:,[2,4]] #take the columns we need
            matches.insert(0,'Summoner Name', str(summoner), True)
            matches.columns = ['Summoner Name','Result','Score']
            
            match_history.append(matches)
            
    return match_history

In [14]:
#the function create_dataframe only create a df without separating the info, we do that here
def df_cleaning(df):
    results = [item.lstrip() for item in df['Result'] if type(item) == str]
    score = [item.lstrip() for item in df['Score'] if type(item) == str]
    status = []
    date = []
    duration= []
    kda =[]
    cs = []
    k = []
    d=[]
    a=[]
    
    s=[]
    
    print(f"Cleaning and  creating columns for {df['Summoner Name'].iloc[0]} data...")
    logging.debug(f"Cleaning and  creating columns for {df['Summoner Name'].iloc[0]} data...")
    
    for result,score in zip(results,score):
        
        if (dt := re.findall(r'\d+ days ago',result)):
            dt = datetime.datetime.now().date() - datetime.timedelta(days=int(re.findall(r'\d+',dt[0])[0]))
            date.append(dt)
            
        elif (dt := re.findall(r'a day ago',result)):
            dt = datetime.datetime.now().date() - datetime.timedelta(days = 1)
            date.append(dt)
    
        else:
            dt = datetime.datetime.now().date()
            date.append(dt)
            
        print(result, score)      
        status.append(re.findall(r'^(Defeat|Victory|Remake)',result)[0])
        duration.append(int(re.findall(r'\d+',re.findall(r'\d+min',result)[0])[0]))           
        s.append(re.findall(r'\d+\s+/\s+\d+\s+/\s+\d+',score)[0])
        cs.append(int(re.findall(r'\d+',re.findall(r'\d+ CS',score)[0])[0]))                   
   

    for item in s:
        temp = re.findall(r'\d+',item)
        x=int(temp[0])
        y=int(temp[1])
        z=int(temp[2])
        k.append(x)
        d.append(y)
        a.append(z)
        if y ==0:        
            kda.append(round(((x+z)/1),2))
        else:
            kda.append(round(((x+z)/y),2))
    
    new_df = pd.DataFrame(list(zip(status,date,duration,k,d,a,kda,cs))) 
    new_df.insert(0,'Summoner Name', df['Summoner Name'], True)
    new_df.columns = ["Summoner Name","W/L", "Date" , "Duration (min)" ,"K" ,"D","A","KDA", "CS"]
    new_df.drop(new_df[new_df['W/L'] == "Remake"].index, inplace = True)
    new_df = new_df.reset_index(drop=True)
    return new_df

In [15]:
#function that takes a list of dataframes, and a list of names and saves each data frame with the names
def save_dataframes(dataframes, names):
    for dataframe, name in zip(dataframes, names):
        if ' ' in name:
            name = re.sub(' ','_',name)           
        dataframe.to_csv(f'match_histories/{name}_flex_match_history.csv', index = False)

In [16]:
def get_matchistories(summoners):
    match_history = create_dataframe(summoners)
    clean_match_history = []
    df = pd.DataFrame()

    for match in match_history: #cleaning dataframes
        m = df_cleaning(match)
        clean_match_history.append(m)
        df = df.append(m).reset_index(drop=True)
     
    try: #saving those dataframes
        save_dataframes(clean_match_history, summoners)
        logging.info('Dataframes saved!')
        df.to_csv(f'match_histories/all_flex_match_history.csv', index = False)
    except:
        logging.exception('An Exception ocurred:')

# And it all comes down to this

In [18]:
summoners = ["Lopso","Cosa","WeIinton Quiw", "Rudon", "El Brayayin", "Eipa",
             "Don Miguel", "Aggression", "DiaCraOne", "IvanRomeroEnFB", "William Kurt", 
             "SexyMuffin", "T O X I C B O Y", "5568882336", "sneik9210", "lumaxile", 
             "skarskarSkarner", "Lixiu", "De Dya", "Rage", "EI ritardo", "Ricky lml"]

In [19]:
#recibe una lista
get_matchistories(summoners)

Lopso, Exception found Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[class="see_more_ajax_button"]"}
  (Session info: MicrosoftEdge=87.0.664.41)

Cosa, Exception found Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[class="see_more_ajax_button"]"}
  (Session info: MicrosoftEdge=87.0.664.41)

WeIinton Quiw, Exception found Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[class="see_more_ajax_button"]"}
  (Session info: MicrosoftEdge=87.0.664.41)

Rudon, Exception found Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[class="see_more_ajax_button"]"}
  (Session info: MicrosoftEdge=87.0.664.41)

El Brayayin, Exception found Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[class="see_more_ajax_button"]"}
  (Session info: MicrosoftEdge=87.0.664.41)

Eipa, E

Victory Flex 499 days ago 24min 40s 5  /  5  /  3  71 CS - 21% Kills P.
Victory Flex 499 days ago 20min 53s 1  /  4  /  11  90 CS - 44% Kills P.
Victory Flex 499 days ago 30min 1s 2  /  7  /  9  96 CS - 30% Kills P.
Defeat Flex 499 days ago 20min 31s 1  /  5  /  7  16 CS - 67% Kills P.
Victory Flex 499 days ago 31min 59s 1  /  9  /  14  22 CS - 38% Kills P.
Defeat Flex 500 days ago 15min 36s 0  /  3  /  3  78 CS - 50% Kills P.
Victory Flex 500 days ago 31min 27s 6  /  5  /  8  135 CS - 39% Kills P.
Defeat Flex 500 days ago 18min 34s 1  /  5  /  0  68 CS - 11% Kills P.
Victory Flex 500 days ago 34min 34s 4  /  4  /  12  143 CS - 46% Kills P.
Victory Flex 500 days ago 36min 9s 8  /  5  /  10  148 CS - 53% Kills P.
Victory Flex 500 days ago 25min 40s 6  /  8  /  11  107 CS - 44% Kills P.
Victory Flex 500 days ago 22min 37s 4  /  4  /  7  97 CS - 42% Kills P.
Victory Flex 501 days ago 37min 41s 9  /  9  /  9  116 CS - 36% Kills P.
Victory Flex 501 days ago 24min 35s 1  /  8  /  7  115 CS -

Cleaning and  creating columns for William Kurt data...
Defeat Flex 2 days ago 27min 27s 6  /  6  /  3  100 CS - 56% Kills P.
Defeat Flex 2 days ago 27min 3s 2  /  8  /  3  144 CS - 42% Kills P.
Defeat Flex 2 days ago 27min 8s 6  /  7  /  7  185 CS - 42% Kills P.
Victory Flex 2 days ago 28min 4s 3  /  3  /  8  186 CS - 27% Kills P.
Defeat Flex 2 days ago 22min 24s 5  /  6  /  2  122 CS - 54% Kills P.
Defeat Flex 3 days ago 17min 41s 1  /  6  /  1  81 CS - 20% Kills P.
Victory Flex 3 days ago 30min 59s 10  /  6  /  5  221 CS - 35% Kills P.
Defeat Flex 3 days ago 36min 22s 11  /  7  /  6  185 CS - 50% Kills P.
Defeat Flex 3 days ago 24min 33s 4  /  10  /  1  138 CS - 36% Kills P.
Victory Flex 3 days ago 32min 33s 1  /  10  /  7  185 CS - 19% Kills P.
Victory Flex 3 days ago 30min 6s 1  /  6  /  6  163 CS - 17% Kills P.
Defeat Flex 8 days ago 24min 31s 1  /  8  /  2  136 CS - 23% Kills P.
Defeat Flex 49 days ago 24min 8s 3  /  7  /  2  133 CS - 33% Kills P.
Victory Flex 49 days ago 30min 

Defeat Flex 59 days ago 24min 41s 3  /  1  /  17  32 CS - 63% Kills P.
Defeat Flex 59 days ago 23min 40s 1  /  5  /  17  24 CS - 51% Kills P.
Defeat Flex 66 days ago 42min 10s 4  /  5  /  27  50 CS - 63% Kills P.
Defeat Flex 66 days ago 35min 55s 1  /  5  /  19  57 CS - 63% Kills P.
Defeat Flex 66 days ago 19min 56s 11  /  3  /  12  139 CS - 51% Kills P.
Defeat Flex 66 days ago 15min 48s 2  /  4  /  7  26 CS - 64% Kills P.
Cleaning and  creating columns for Ricky lml data...
Victory Flex 3 days ago 32min 49s 16  /  7  /  7  200 CS - 48% Kills P.
Victory Flex 91 days ago 75min 9s 15  /  1  /  42  157 CS - 59% Kills P.
Victory Flex 123 days ago 23min 43s 7  /  2  /  7  168 CS - 44% Kills P.
Defeat Flex 123 days ago 28min 29s 5  /  2  /  9  173 CS - 48% Kills P.
Victory Flex 123 days ago 40min 42s 15  /  5  /  7  240 CS - 34% Kills P.
Defeat Flex 123 days ago 18min 28s 2  /  4  /  2  76 CS - 31% Kills P.
Defeat Flex 123 days ago 20min 53s 7  /  0  /  4  114 CS - 61% Kills P.
Defeat Flex 1